In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### Neural Network Architecture
<img src="https://pytorch.org/tutorials/_images/mnist.png">

In [2]:
class Net(nn.Module):
    """
    For created a network, you only have to define the forward propagation.
    Backward propagation is handled by autograd
    """
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # Flatten x got the fully-connected layers
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        """
        Gets how many raw elements are in a matrix (for flattening it)
        """
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
params = list(net.parameters())
print(len(params))

# conv1's weights (size)
# Makes sense because there are 6 filters with 3x3 kernel size
print(params[0].size()) 

10
torch.Size([6, 1, 3, 3])


In [5]:
# Random input test (Note: PyTorch only accepts mini-batches, so we add a dimension in the beggining as a batch dimension)
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0433, -0.1033,  0.1572,  0.0290,  0.0468, -0.1073, -0.0578,  0.0260,
         -0.0888,  0.0094]], grad_fn=<AddmmBackward>)


In [6]:
# Before getting gradients, we first must zero the buffers of all the parameter gradeients
net.zero_grad()

# This just performs backpropagation with random gradients
out.backward(torch.randn(1, 10))

### Loss

In [21]:
output = net(input)
# A dummy target (as in what the last layer of the neural network should be)
target = torch.randn(10)

# Resize to be the same dimensions of the output layer (1, 10)
target = target.view(1, -1)

# The loss function
criterion = nn.MSELoss()

# Use the loss function
loss = criterion(output, target)
print(loss)

tensor(1.8626, grad_fn=<MseLossBackward>)


In [20]:
# Zero the gradient buffers of all the parameters
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

# loss.backword() differentiates the graph with respect to the loss
# All the Tensors in the graph that had have requires_grad=True will have their .grad Tensor accumulated with the gradient
loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0044,  0.0069,  0.0047,  0.0032,  0.0030,  0.0033])


### Optimization

In [24]:
# Create the optimizer
opt = optim.SGD(net.parameters(), lr=0.01)

# Within a traning loop:
opt.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
# Does the update
opt.step()